# Tools

In [ ]:
from app import Application
from array import array
from blocks import transform
from blocks.bbs import BBS
from blocks.acm import ACM
from blocks.rt import RT
from errors import ValidationError
from PIL import Image
from suplementary import number_theory as nt
from time import time

import numpy as np
import math as mt
import pandas as pd
import sys


primes = array('I')
a = open('primes/100000.txt', 'r')
lines = a.readlines()
for line in lines[5:]:
    for p in line[:-2].split():
        p = int(p)
        if p % 4 == 3 and p < 65536:
            primes.append(int(p))


def generate_s(m):
    while True:
        ret = np.random.randint(2, m, dtype=np.int64)
        if mt.gcd(ret, m) == 1:
            return ret


def next_s(current, m):
    for lanjut in range(current + 1, m):
        if mt.gcd(lanjut, m) == 1:
            return lanjut
    return m


def attack(plainfile, cipherimage, a, b, n):
    start = time()

    # ENCRYPT
    ## TRANSFORMATION
    plainbytes = transform.buffer_to_bytes(plainfile)
    plainbytes = transform.pad_bytes(plainbytes)

    # DECRYPT
    ## TRANSFORMATION
    ciphermatrix = transform.image_to_matrix(cipherimage)

    ## ARNOLD'S CAT MAP
    plainmatrix = ACM(a, b, n).decrypt(ciphermatrix)

    ## TRANSFORMATION
    cipherbytes = transform.matrix_to_bytes(plainmatrix)


    # SEARCHING FOR p, q, and s
    i = 0
    found = False
    while i < len(primes) - 1 and not found:
        p_g = primes[i]
        j = i + 1
        while j < len(primes) and not found:
            q_g = primes[j]
            m_g = p_g * q_g
            s_g = next_s(1, m_g)
            while s_g < m_g and not found:
                bbs = BBS(p_g, q_g, s_g)
                matched = True
                i_p = 0
                while i_p < plainbytes.size and matched:
                    key = bbs.next()
                    plainbytes_guess = nt.mod_add(
                        cipherbytes[2 * i_p] - cipherbytes[2 * i_p + 1],
                        key,
                        256
                    )
                    if plainbytes_guess != plainbytes[i_p]:
                        matched = False
                    i_p += 1
                if matched:
                    found = True
                else:
                    s_g = next_s(s_g, m_g)
            j += 1
        i += 1

    waktu = time() - start
    plainfile_f = Application(p_g, q_g, s_g, a, b, n).decrypt(cipherimage)
    if plainfile == plainfile_f:
        return waktu, (p_g, q_g, s_g)
    else:
        return None, (None, None, None)

# Performance Testing

In [ ]:
dt = []

a = 1
b = 1
n = 42
B = 321000

for i in np.linspace(1, len(primes)-1, 30):
    i = mt.ceil(i)
    p = primes[0]
    q = primes[1]
    s = generate_s(p * q)
    
    print(p, q, s)

    plainfile = (170).to_bytes(1, 'little') * B
    cipherimage = Application(p, q, s, a, b, n).encrypt(plainfile)
    waktu, key = attack(plainfile, cipherimage, a, b, n)
    
    dt.append([waktu, p, q, s] + list(key))

df = pd.DataFrame(dt)
df.to_csv("kpa.csv", index=False)
print('Done')

## SET UP

In [ ]:
# len(primes) = 3284

# UNKOWN
p = primes[0]
q = primes[1]
s = generate_s(p * q)
print('These value are secret:\n\tp={0},\n\tq={1},\n\ts={2}'.format(p, q, s))

# KNOWN
a = 1
b = 1
n = 42
B = 100
print('These value are no more a secret:\n\ta={0},\n\tb={1},\n\tn={2},\n\tB={3}'.format(a, b, n, B))

# Construct `plainfile` and `cipherimage`

In [ ]:
plainfile = np.random.randint(0, 256, B, 'B').tobytes()
cipherimage = Application(p, q, s, a, b, n).encrypt(plainfile)

## Attack

In [ ]:
attack(plainfile, cipherimage, a, b, n)